In [1]:
from bayesian_statistics.model3_config import Model3Config
from bayesian_statistics.model3_preprocessing import ObsidianDataPreprocessor

config = Model3Config(
    data_dir="/home/ohta/dev/bayesian_statistics/data/",
)

# 前処理クラスのインスタンス化
preprocessor = ObsidianDataPreprocessor(config.data_dir)
preprocessor.load_data()

# データの読み込み
data = preprocessor.create_X_Y(
    config.nw_variable_names, config.time_periods, config.origins
)

X = data["X"]
Y = data["Y"]
X_columns = data["X_columns"]
Y_columns = data["Y_columns"]

X_test = data["X_test"]

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
Y_columns

['count_0_神津島',
 'count_0_信州',
 'count_0_箱根',
 'count_0_高原山',
 'count_1_神津島',
 'count_1_信州',
 'count_1_箱根',
 'count_1_高原山',
 'count_2_神津島',
 'count_2_信州',
 'count_2_箱根',
 'count_2_高原山',
 'count_3_神津島',
 'count_3_信州',
 'count_3_箱根',
 'count_3_高原山',
 'count_4_神津島',
 'count_4_信州',
 'count_4_箱根',
 'count_4_高原山']

In [3]:
Y[:, 0].sum(), Y[:, 1].sum(), Y[:, 2].sum(), Y[:, 3].sum()

(1019, 342, 90, 15)

In [4]:
Y_0 = Y[:, :4]
Y_1 = Y[:, 4:8]
Y_2 = Y[:, 8:12]
Y_3 = Y[:, 12:16]
Y_4 = Y[:, 16:20]

Y_0.shape, Y_1.shape, Y_2.shape, Y_3.shape, Y_4.shape

((274, 4), (274, 4), (274, 4), (274, 4), (274, 4))

In [5]:
Y_0_sum = Y_0.sum(axis=1)
Y_1_sum = Y_1.sum(axis=1)
Y_2_sum = Y_2.sum(axis=1)
Y_3_sum = Y_3.sum(axis=1)
Y_4_sum = Y_4.sum(axis=1)

In [6]:
print("X_columns:")
print(X_columns)
print(f"\nX shape: {X.shape}")
print(f"Number of X_columns: {len(X_columns)}")
print("\nFirst few rows of X:")
print(X[:5])

X_columns:
['lon', 'lat', 'average_elevation', 'average_slope_angle', 'cost_kouzu', 'cost_shinshu', 'cost_hakone', 'cost_takahara', 'cost_river']

X shape: (274, 9)
Number of X_columns: 9

First few rows of X:
[[6.38014994e-01 2.41910732e+00 7.66700000e+02 1.17000000e+01
  5.47487486e+03 1.59746539e+03 3.80995697e+03 3.30166806e+03
  1.12847016e+01]
 [6.38050185e-01 2.41945701e+00 7.44100000e+02 1.35000000e+01
  5.44931120e+03 1.62856470e+03 3.78439332e+03 3.27610440e+03
  0.00000000e+00]
 [6.38687997e-01 2.42277639e+00 4.04700000e+02 9.50000000e+00
  5.00201054e+03 2.14982395e+03 3.33709265e+03 2.82746383e+03
  5.50114679e+00]
 [6.32966008e-01 2.42293785e+00 2.21500000e+02 1.09000000e+01
  4.44207779e+03 1.71835047e+03 2.77715991e+03 2.41196822e+03
  0.00000000e+00]
 [6.33890835e-01 2.42339581e+00 2.09300000e+02 3.60000000e+00
  4.50263351e+03 1.66890505e+03 2.83771562e+03 2.35090780e+03
  0.00000000e+00]]
